In [27]:
import pandas as pd
import numpy as np
import re

pd.set_option('max_colwidth', 100)
pd.set_option('display.width', 2000)

In [3]:
file1 = open('../data/train.txt', 'r') 
lines = file1.readlines()

In [4]:
ids = []
revs = []
comps = []
txts = []

for line in lines:
    ids.append(line[1:5])
    revs.append(line[6:9])
    comps.append(line[10:13])
    txts.append(line[15:])

In [5]:
df = pd.DataFrame({"Avis" : revs, "Entreprise" : comps, "Tweet" : txts}, index = ids)
df.head(10)

,Avis,Entreprise,Tweet
0000,neu,apl,20 min line @apple store @short pump.\n
0001,irr,msf,Nueva tecnología convierte cualquier superfici...
0002,neu,ggl,Some people should not post replies in #Google...
0003,neg,apl,I know a few others having same issue RT @Joel...
0004,neg,msf,"#Microsoft - We put the """"backwards"""" into bac..."
0005,neg,twt,#twitter is sooo trash ritenow with all dezz #...
0006,neu,apl,RT @jesperordrup: Hi @apple. Household has 4 i...
0007,irr,msf,#ALG Culminando formación en #Microsoft M-2778...
0008,neu,msf,#Microsoft Community Blogs The 7/365 Review - ...
0009,irr,twt,Buenas noches a todos #Twitter off\n


In [6]:
df.Avis.unique()

array(['neu', 'irr', 'neg', 'pos'], dtype=object)

In [7]:
df.Entreprise.unique()

array(['apl', 'msf', 'ggl', 'twt'], dtype=object)

In [8]:
len(df)

4173

In [9]:
df.groupby(by="Avis").count()

,Entreprise,Tweet
Avis,,
irr,1389,1389
neg,486,486
neu,1871,1871
pos,427,427


In [10]:
df.groupby(by=["Entreprise", "Avis"]).count()

Tweet
Entreprise Avis       
apl        irr     125
           neg     267
           neu     422
           pos     140
ggl        irr     398
           neg      47
           neu     455
           pos     167
msf        irr     401
           neg     111
           neu     516
           pos      73
twt        irr     465
           neg      61
           neu     478
           pos      47

In [29]:
df_with_hashtags = df[df.Tweet.str.contains("#")]
print(len(df_with_hashtags))
df_with_hashtags.head(10)

3583


,Avis,Entreprise,Tweet
0001,irr,msf,Nueva tecnología convierte cualquier superficie en una pantalla multitactil. http://t.co/EDibLL5...
0002,neu,ggl,Some people should not post replies in #Google+ threads. Their posts only continue to weaken the...
0004,neg,msf,"#Microsoft - We put the """"backwards"""" into backwards compatibility. #instantfollowback\n"
0005,neg,twt,#twitter is sooo trash ritenow with all dezz #highscoolmemories -__-\n
0007,irr,msf,#ALG Culminando formación en #Microsoft M-2778 y en #IndicadoresDeGestión. Gracias a nuestros cl...
0008,neu,msf,#Microsoft Community Blogs The 7/365 Review - The Cloud's Impact on Business http://t.co/gPB2MjFW\n
0009,irr,twt,Buenas noches a todos #Twitter off\n
0010,neu,apl,This video brought a small tear to my eye http://t.co/3va2ninp. Watch to @SteveWoz opinion of #S...
0011,neg,msf,"I gave a try to Windows 8 Dev. Preview and it was good but the UI is way too messy right now, wa..."
0012,neu,ggl,now watching live stream of #Google new mobile #Nexus\n


In [26]:
df_with_mentions = df[df.Tweet.str.contains("@")]
print(len(df_with_mentions))
df_with_mentions.head(10)

1908


,Avis,Entreprise,Tweet
0000,neu,apl,20 min line @apple store @short pump.\n
0003,neg,apl,I know a few others having same issue RT @Joelplane: 9% now on my second full charge of the day....
0006,neu,apl,"RT @jesperordrup: Hi @apple. Household has 4 iphones, 2 ipads, 2 minis, 2 apple tv, AirPorts, Ti..."
0010,neu,apl,This video brought a small tear to my eye http://t.co/3va2ninp. Watch to @SteveWoz opinion of #S...
0014,pos,apl,@apple..good ipad\n
0015,neu,apl,@BikerVox @mariomussolini @ios_notch @apple xD thats why i have a back up PC… just incase *hides...
0017,pos,msf,"@RAD_Software Yes good points about #ERP but unfair assessment in saying """"#SAGE, #SAP and #Micr..."
0023,neg,twt,RT @nagoul1: #Retweets section of my #twitter account has not been working for 17 hours! Is Ever...
0025,neg,apl,"Comparing @Office #live with @google #apps and @apple #iCloud , I do not see #iCloud leading the..."
0026,irr,ggl,@iley_little we'lll figure it out tm in gym #google\n


In [365]:
#hashtag_re = r"#([^\s,]*)"
hashtag_re = r"#([\w\+_-]*)"
mention_re = r"@(\w*)"
link_re = r"(https?://|(https?://)?bit.ly)[^\s]+"

#preprocessing
def remove_slash_n(tweet):
    return tweet[:-1] if(tweet.endswith("\n")) else tweet
    
def replace_links(tweet, rep="_LINK_"):
    return re.sub(link_re, rep, tweet)

def remove_hashtags_and_mentions(tweet):
    s = re.sub(hashtag_re, r"\1", tweet)
    return re.sub(mention_re, r"\1", s)

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "(" + (UNICODE_EMO[emot].replace(",","").replace(":","")\
                                                .replace("_"," ")) + ")")
    return text

def preprocess_tweet(tweet):
    return convert_emojis(replace_links(remove_slash_n(remove_hashtags_and_mentions(tweet))))\
            if tweet != None else None

#finding list of hashtags & mentions
def find_hashtags(tweet):
    return [h for h in re.findall(hashtag_re, tweet) if h != ""]

def find_mentions(tweet):
    return [h for h in re.findall(mention_re, tweet) if h != ""]

In [347]:
text1 = "👦👧👩👨👶👵👴👱👲👳👷💂👸👮..... Um why No black people❔❔❔Apple"
convert_emojis(text1)

'(boy)(girl)(woman)(man)(baby)(old woman)(old man)(blond-haired person)(man with Chinese cap)(person wearing turban)(construction worker)(guard)(princess)(police officer)..... Um why No black people(white question mark)(white question mark)(white question mark)Apple'

In [144]:
re.findall(r'@([^\s,]*)', "hello @worldéç/ù, @Test")

['worldéç/ù', 'Test']

In [145]:
re.findall(hashtag_re, "hello #world-é#ç/ù, #Test")

['world-é', 'ç', 'Test']

In [146]:
re.sub(r"https?://[^\s]+", "_LINK_", "http://t.co/3va2ninp. hello @worldéçù @Test")

'_LINK_ hello @worldéçù @Test'

In [147]:
re.sub(r"@(\w*)", r"\1", "hello @worldéç/ù @Test")

'hello worldéç/ù Test'

In [148]:
preprocess_tweet("20 min line @apple store @short pump.")

'20 min line apple store short pump.'

In [366]:
df = pd.read_pickle("../data/df.pk")

df["Hashtags"] = df.Tweet.apply(find_hashtags)
df["Mentions"] = df.Tweet.apply(find_mentions)
df["Tweet"] = df.Tweet.apply(preprocess_tweet)

In [367]:
df["Language"] = df.Tweet.apply(lambda x :
                                model.predict(tweet_to_lower(x), k=1)[0][0][len("__label__"):])
df["LanguageProbability"] = df.Tweet.apply(lambda x :
                                model.predict(tweet_to_lower(x), k=1)[1][0])

In [368]:
df.head(10)

,Avis,Entreprise,Tweet,Hashtags,Mentions,Language,LanguageProbability
0000,neu,apl,20 min line apple store short pump.,[],"[apple, short]",en,0.670655
0001,irr,msf,Nueva tecnología convierte cualquier superficie en una pantalla multitactil. _LINK_ Microsoft om...,"[Microsoft, omnitouch]",[],es,0.876679
0002,neu,ggl,Some people should not post replies in Google+ threads. Their posts only continue to weaken thei...,[Google+],[],en,0.989970
0003,neg,apl,I know a few others having same issue RT Joelplane: 9% now on my second full charge of the day. ...,[],"[Joelplane, Apple]",en,0.973644
0004,neg,msf,"Microsoft - We put the """"backwards"""" into backwards compatibility. instantfollowback","[Microsoft, instantfollowback]",[],en,0.835478
0005,neg,twt,twitter is sooo trash ritenow with all dezz highscoolmemories -__-,"[twitter, highscoolmemories]",[],en,0.695345
0006,neu,apl,"RT jesperordrup: Hi apple. Household has 4 iphones, 2 ipads, 2 minis, 2 apple tv, AirPorts, Time...",[],"[jesperordrup, apple]",en,0.629808
0007,irr,msf,ALG Culminando formación en Microsoft M-2778 y en IndicadoresDeGestión. Gracias a nuestros clien...,"[ALG, Microsoft, IndicadoresDeGestión]",[],es,0.912664
0008,neu,msf,Microsoft Community Blogs The 7/365 Review - The Cloud's Impact on Business _LINK_,[Microsoft],[],en,0.760493
0009,irr,twt,Buenas noches a todos Twitter off,[Twitter],[],es,0.550812


In [369]:
df.to_pickle("../data/df_preproc.pk")

In [297]:
import fasttext
model = fasttext.load_model('../data/lid.176.bin')

model.predict('a child’s-eye view of microsoft’s kinect for kids: _LINK_ | solutions kasper', k=1)

(('__label__en',), array([0.66514164]))

In [359]:
def tweet_to_lower(tw):
    return tw.lower().replace("microsoft", "Microsoft").replace("google", "Google")\
        .replace("apple", "Apple").replace("twitter", "Twitter")

tweet_to_lower("Nueva tecnología convierte cualquier superficie en una pantalla multitactil. _LINK_ Microsoft ")

'nueva tecnología convierte cualquier superficie en una pantalla multitactil. _link_ Microsoft '

In [353]:
df[df.Avis == "irr"].groupby(by=["Language"])["Tweet"].count().sort_values(ascending=False)

Language
es     539
en     185
pt     140
fr     125
nl      96
de      85
ru      44
ja      39
it      33
ar      21
id      13
sv       9
tr       6
cs       5
zh       4
pl       4
th       4
ro       3
no       3
el       2
gl       2
arz      2
bs       2
da       2
fi       2
sw       2
he       2
sk       1
gom      1
mn       1
tl       1
su       1
sr       1
cy       1
hu       1
ca       1
ur       1
bg       1
bar      1
ast      1
mk       1
la       1
Name: Tweet, dtype: int64

In [354]:
df[df.Avis != "irr"].groupby(by=["Language"])["Tweet"].count().sort_values(ascending=False)

Language
en     2721
es       15
pt       13
it        8
de        5
ja        5
fr        4
la        3
uk        2
zh        1
fi        1
jbo       1
nl        1
ru        1
sv        1
tr        1
ceb       1
Name: Tweet, dtype: int64

In [355]:
df[(df.Avis != "irr") & (df.Language != "en")]

,Avis,Entreprise,Tweet,Hashtags,Mentions,Language,LanguageProbability
0049,neu,ggl,_LINK_ google,[google],[],pt,0.305487
0144,neu,twt,bye bye twitter ;),[twitter],[],ja,0.191677
0147,neu,twt,I OUT Twitter Till 2mar,[Twitter],[],pt,0.137228
0166,pos,ggl,Tita_Ramos Google it =),[Google],[Tita_Ramos],es,0.353690
0201,neu,ggl,"Google Galaxy Nexus, Miam ! _LINK_","[Google, Galaxy, Nexus]",[],la,0.241467
0406,neu,apl,Apple MacBook Air.. satiliktir.,[],[Apple],tr,0.539347
0463,neu,ggl,Android Google Galaxy Nexus _LINK_ DhilipSiva,"[Android, Google, DhilipSiva]",[],it,0.107048
0480,neu,ggl,Google Samsung GalaxyNexus _LINK_,"[Google, Samsung, GalaxyNexus]",[],pt,0.124567
0497,neu,ggl,Samsung e o Google lançam hoje o Galaxy Nexus e o Android 4.0 _LINK_ google samsung,"[google, samsung]",[],pt,0.741312
0513,neu,twt,Twitter FDP,[Twitter],[],fr,0.137798


In [356]:
df.head()

,Avis,Entreprise,Tweet,Hashtags,Mentions,Language,LanguageProbability
0000,neu,apl,20 min line apple store short pump.,[],"[apple, short]",en,0.670655
0001,irr,msf,Nueva tecnología convierte cualquier superficie en una pantalla multitactil. _LINK_ Microsoft om...,"[Microsoft, omnitouch]",[],es,0.876679
0002,neu,ggl,Some people should not post replies in Google+ threads. Their posts only continue to weaken thei...,[Google+],[],en,0.989970
0003,neg,apl,I know a few others having same issue RT Joelplane: 9% now on my second full charge of the day. ...,[],"[Joelplane, Apple]",en,0.973644
0004,neg,msf,"Microsoft - We put the """"backwards"""" into backwards compatibility. instantfollowback","[Microsoft, instantfollowback]",[],en,0.835478


In [358]:
def irr_classification_by_lang_acc(df, prob_threshold):
    return (len(df[(df.Avis != "irr") & ((df.Language == "en") | (df.LanguageProbability <= prob_threshold))]) +\
        len(df[(df.Avis == "irr") & (df.Language != "en") & (df.LanguageProbability > prob_threshold)]))/len(df)

irr_classification_by_lang_acc(df, 0)

0.9405703330936975

In [360]:
df[df.Avis == "neu"]

,Avis,Entreprise,Tweet,Hashtags,Mentions,Language,LanguageProbability
0000,neu,apl,20 min line apple store short pump.,[],"[apple, short]",en,0.670655
0002,neu,ggl,Some people should not post replies in Google+ threads. Their posts only continue to weaken thei...,[Google+],[],en,0.989970
0006,neu,apl,"RT jesperordrup: Hi apple. Household has 4 iphones, 2 ipads, 2 minis, 2 apple tv, AirPorts, Time...",[],"[jesperordrup, apple]",en,0.629808
0008,neu,msf,Microsoft Community Blogs The 7/365 Review - The Cloud's Impact on Business _LINK_,[Microsoft],[],en,0.760493
0010,neu,apl,This video brought a small tear to my eye _LINK_ Watch to SteveWoz opinion of SteveJobs and Apple,[SteveJobs],"[SteveWoz, Apple]",en,0.835962
0012,neu,ggl,now watching live stream of Google new mobile Nexus,"[Google, Nexus]",[],en,0.674474
0013,neu,twt,"Dear twitter, Now that we all know I'm awesome could you write me a new """"all about me"""" for my ...",[twitter],[],en,0.946373
0015,neu,apl,BikerVox mariomussolini ios_notch apple xD thats why i have a back up PC… just incase *hides gun*,[],"[BikerVox, mariomussolini, ios_notch, apple]",en,0.504904
0016,neu,ggl,Google Android IceCreamSandwich features - _LINK_ ICS,"[Google, Android, IceCreamSandwich, ICS]",[],en,0.653969
0018,neu,twt,These Hoes Follow Me Like Twitter !,[Twitter],[],en,0.705591


In [361]:
df[(df.Avis == "irr") & (df.apply(lambda x : ((x["Tweet"].lower().find("google") == -1) and
                                              (x["Tweet"].lower().find("apple") == -1) and
                                              (x["Tweet"].lower().find("microsoft") == -1) and
                                              (x["Tweet"].lower().find("twitter") == -1)), axis=1))]

,Avis,Entreprise,Tweet,Hashtags,Mentions,Language,LanguageProbability
1744,irr,twt,RT alisalam1990: البايو في تويتر مثل عناوين المنازل لا تكشف عن صاحبها إلا إذا دخلت التايم لاين ،...,[],[alisalam1990],ar,0.997204
1890,irr,ggl,"RT juank3946: IceCreamSandwich presenta AndroidBeam, una herramienta para transferir archivos co...","[IceCreamSandwich, AndroidBeam, Android, G]",[juank3946],es,0.839607
2004,irr,msf,RT ZiOu333: Et Sony se fait encore pirater !! C'est la fête du slip chez eux. Faudrai peut-être ...,"[Sony, Micr]",[ZiOu333],fr,0.973054
2143,irr,msf,"RT GroenewegBV: Steve Ballmer haalt uit naar Android _LINK_ nuandroid Dan ben ik ook een nerd, j...","[Android, nuandroid]",[GroenewegBV],nl,0.986176
2977,irr,msf,RT AslanMarine: Vous voulez la une de LNT ? La nouvelle tribune de demain 20/10/11: intikhabates...,"[intikhabates, PJD, PPS, MoroccoMall, Boeing]",[AslanMarine],fr,0.642571
3629,irr,msf,"RT Brittjje: Morgen maak ik bekend wie een WindowsPhone7.5 krijgt, is ook de laatste dag dat je ...",[],[Brittjje],nl,0.982220


In [336]:
from emot.emo_unicode import UNICODE_EMO#, EMOTICONS